In [47]:
import pandas as pd
import numpy as np
import re

import html
from unicodedata import normalize

import string

import nltk
from nltk.stem import WordNetLemmatizer

import sys
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.parse.malt import MaltParser
from nltk.corpus import words
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/gerry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gerry/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/gerry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/gerry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [48]:
reddit_df = pd.read_csv("../chatgpt-reddit-comments.csv")

In [49]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT
3,3,iztfhtb,t3_zj2aeu,"Worked on me, ngl.",r/ChatGPT
4,4,izu2as9,t3_zj2aeu,"Certified 10/10, must-see moment. It really di...",r/ChatGPT
...,...,...,...,...,...
52411,16668,j5m0v6m,t3_10jmvpj,"Read the T.O.S., you'll thank me later",r/technology
52412,16669,j5m6aj0,t1_j5m0v6m,What am I missing here... https://openai.com/t...,r/technology
52413,16670,j5nylax,t1_j5m0v6m,What does ChatGTP think of its own TOS?,r/technology
52414,16671,j5mwpdr,t1_j5m6aj0,Don't know what they're referring to in the TO...,r/technology


In [50]:
academic_words_list = ["class", "lecture", "professor", "prof", "homework", "hw", "exam", "examinations", "assignment", 
"study", "degree", "gpa", "cap", "scholarship", "scholar", "research", "thesis", "lab", "campus", "graduation", "grad", "syllabus",
"textbook", "textbooks", "student", "students", "academic", "acad", "acads", "registrar", "tuition", "coursework", "course", "attendance",
"faculty", "teacher", "teach", "learn", "internship", "intern", "library", "peer", "peers", "school", "schools", "university", "uni", "college", "colleges"
]

In [51]:
"""
Code references:
    https://pythonguides.com/remove-unicode-characters-in-python/
    https://www.kite.com/python/answers/how-to-decode-html-entities-in-python
"""
def decode_text(text):
    # remove non-ASCII characters in string
    decoded_text = text.encode('ascii', 'ignore').decode('utf-8')

    # decode HTML entities
    decoded_html = html.unescape(decoded_text)
    return ''.join([word for word in decoded_html if word.isprintable()])

"""
Code reference:
    https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
"""
def remove_links(text):
    return re.sub(r"http\S+", "", text)

"""
Code reference:
    https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/
"""
def remove_mentions(text):
    return re.sub("@[A-Za-z0-9_]+","", text)

def remove_hashtags(text):
    return re.sub("#[A-Za-z0-9_]+","", text)

def remove_stopwords(words_list):
    stop_list = stopwords.words("english")
    stop_list.append("filler")
    return [word for word in words_list if word not in stop_list]

def pos_to_wordnet(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_words(word_list):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    # POS (part-of-speech) tagging
    # nltk_tagged -> a list of tuples (word, pos tag)
    nltk_tagged = nltk.pos_tag(word_list)

    # returns a list of tuples of words and their wordnet_tag (after conversion from NLTK tag)
    wordnet_tagged = list(map(lambda x: (x[0], pos_to_wordnet(x[1])), nltk_tagged))

    # lemmatizing
    lemmatized_words = []
    for word, tag in wordnet_tagged:
        if tag is not None:
            # need POS tag as 2nd argument as it helps lemmatize the words more accurately
            lemmatized_words.append(lemmatizer.lemmatize(word, tag))
        elif tag in [wordnet.NOUN]:
            lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def clean_original_text(text):
    text = str(text)
    text = text.lower()
    clean_list = []
    sentence_list = nltk.sent_tokenize(text)
    for sentence in sentence_list:
        decoded_sentence = decode_text(sentence)
        linkless_sentence = remove_links(decoded_sentence)
        mentionless_sentence = remove_mentions(linkless_sentence)
        tagless_sentence = remove_hashtags(mentionless_sentence)
        words_list = nltk.RegexpTokenizer(r'\w+').tokenize(tagless_sentence)
        lemmatized_words = lemmatize_words(words_list)
        useful_words = remove_stopwords(lemmatized_words)

        if len(useful_words) > 0:
            clean_list.extend(useful_words)
    clean_text = ' '.join(clean_list)

    return clean_text

In [52]:
reddit_df["clean_text"] = reddit_df["comment_body"].apply(clean_original_text)
reddit_df["clean_tokens"] = reddit_df["clean_text"].apply(nltk.word_tokenize)

In [53]:
reddit_df = reddit_df.drop_duplicates(subset=["clean_text"])

In [54]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,clean_text,clean_tokens
0,0,iztdxuh,t3_zj2aeu,"I've been shocked for days now, I don't need c...",r/ChatGPT,shock day need clickbait,"[shock, day, need, clickbait]"
1,1,iztn0q0,t3_zj2aeu,\n\nI am so angry right now. I just wasted my...,r/ChatGPT,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c..."
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT,chatgpt karma whoring folk think stream though...,"[chatgpt, karma, whoring, folk, think, stream,..."
3,3,iztfhtb,t3_zj2aeu,"Worked on me, ngl.",r/ChatGPT,work ngl,"[work, ngl]"
4,4,izu2as9,t3_zj2aeu,"Certified 10/10, must-see moment. It really di...",r/ChatGPT,certify see moment really shock core,"[certify, see, moment, really, shock, core]"
...,...,...,...,...,...,...,...
52411,16668,j5m0v6m,t3_10jmvpj,"Read the T.O.S., you'll thank me later",r/technology,read thank later,"[read, thank, later]"
52412,16669,j5m6aj0,t1_j5m0v6m,What am I missing here... https://openai.com/t...,r/technology,missing,[missing]
52413,16670,j5nylax,t1_j5m0v6m,What does ChatGTP think of its own TOS?,r/technology,chatgtp think tos,"[chatgtp, think, tos]"
52414,16671,j5mwpdr,t1_j5m6aj0,Don't know what they're referring to in the TO...,r/technology,know refer tos section seem noteworthy c c c k,"[know, refer, tos, section, seem, noteworthy, ..."


In [55]:
reddit_df = reddit_df.dropna(subset=["comment_body"])

In [73]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,clean_text,clean_tokens,text_contain_academic
0,0,iztdxuh,t3_zj2aeu,"i've been shocked for days now, i don't need c...",r/ChatGPT,shock day need clickbait,"[shock, day, need, clickbait]",False
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c...",True
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT,chatgpt karma whoring folk think stream though...,"[chatgpt, karma, whoring, folk, think, stream,...",False
3,3,iztfhtb,t3_zj2aeu,"worked on me, ngl.",r/ChatGPT,work ngl,"[work, ngl]",False
4,4,izu2as9,t3_zj2aeu,"certified 10/10, must-see moment. it really di...",r/ChatGPT,certify see moment really shock core,"[certify, see, moment, really, shock, core]",False
...,...,...,...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"scary thing is, in this future text won’t be t...",r/technology,scary thing future text wont trustworthy case ...,"[scary, thing, future, text, wont, trustworthy...",False
52411,16668,j5m0v6m,t3_10jmvpj,"read the t.o.s., you'll thank me later",r/technology,read thank later,"[read, thank, later]",False
52412,16669,j5m6aj0,t1_j5m0v6m,what am i missing here... https://openai.com/t...,r/technology,missing,[missing],False
52413,16670,j5nylax,t1_j5m0v6m,what does chatgtp think of its own tos?,r/technology,chatgtp think tos,"[chatgtp, think, tos]",False


In [57]:
reddit_df["comment_body"] = reddit_df["comment_body"].str.lower()

/var/folders/52/p78_6rrn58d3q_81k3cgd9gw0000gn/T/ipykernel_25041/3301605234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df["comment_body"] = reddit_df["comment_body"].str.lower()


In [58]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,clean_text,clean_tokens
0,0,iztdxuh,t3_zj2aeu,"i've been shocked for days now, i don't need c...",r/ChatGPT,shock day need clickbait,"[shock, day, need, clickbait]"
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c..."
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT,chatgpt karma whoring folk think stream though...,"[chatgpt, karma, whoring, folk, think, stream,..."
3,3,iztfhtb,t3_zj2aeu,"worked on me, ngl.",r/ChatGPT,work ngl,"[work, ngl]"
4,4,izu2as9,t3_zj2aeu,"certified 10/10, must-see moment. it really di...",r/ChatGPT,certify see moment really shock core,"[certify, see, moment, really, shock, core]"
...,...,...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"scary thing is, in this future text won’t be t...",r/technology,scary thing future text wont trustworthy case ...,"[scary, thing, future, text, wont, trustworthy..."
52411,16668,j5m0v6m,t3_10jmvpj,"read the t.o.s., you'll thank me later",r/technology,read thank later,"[read, thank, later]"
52412,16669,j5m6aj0,t1_j5m0v6m,what am i missing here... https://openai.com/t...,r/technology,missing,[missing]
52413,16670,j5nylax,t1_j5m0v6m,what does chatgtp think of its own tos?,r/technology,chatgtp think tos,"[chatgtp, think, tos]"


In [74]:
academic_words_list_pattern = '|'.join(academic_words_list)

In [75]:
academic_words_list_pattern

'class|lecture|professor|prof|homework|hw|exam|examinations|assignment|study|degree|gpa|cap|scholarship|scholar|research|thesis|lab|campus|graduation|grad|syllabus|textbook|textbooks|student|students|academic|acad|acads|registrar|tuition|coursework|course|attendance|faculty|teacher|teach|learn|internship|intern|library|peer|peers|school|schools|university|uni|college|colleges'

In [76]:
reddit_df["text_contain_academic"] = reddit_df["comment_body"].str.contains(academic_words_list_pattern)

/var/folders/52/p78_6rrn58d3q_81k3cgd9gw0000gn/T/ipykernel_25041/1998606852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reddit_df["text_contain_academic"] = reddit_df["comment_body"].str.contains(academic_words_list_pattern)


In [77]:
reddit_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,clean_text,clean_tokens,text_contain_academic
0,0,iztdxuh,t3_zj2aeu,"i've been shocked for days now, i don't need c...",r/ChatGPT,shock day need clickbait,"[shock, day, need, clickbait]",False
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c...",True
2,2,izudrph,t3_zj2aeu,chatgpt karma whoring is here folks! just when...,r/ChatGPT,chatgpt karma whoring folk think stream though...,"[chatgpt, karma, whoring, folk, think, stream,...",False
3,3,iztfhtb,t3_zj2aeu,"worked on me, ngl.",r/ChatGPT,work ngl,"[work, ngl]",False
4,4,izu2as9,t3_zj2aeu,"certified 10/10, must-see moment. it really di...",r/ChatGPT,certify see moment really shock core,"[certify, see, moment, really, shock, core]",False
...,...,...,...,...,...,...,...,...
52410,16667,j4lbiix,t1_j4kgcac,"scary thing is, in this future text won’t be t...",r/technology,scary thing future text wont trustworthy case ...,"[scary, thing, future, text, wont, trustworthy...",False
52411,16668,j5m0v6m,t3_10jmvpj,"read the t.o.s., you'll thank me later",r/technology,read thank later,"[read, thank, later]",False
52412,16669,j5m6aj0,t1_j5m0v6m,what am i missing here... https://openai.com/t...,r/technology,missing,[missing],False
52413,16670,j5nylax,t1_j5m0v6m,what does chatgtp think of its own tos?,r/technology,chatgtp think tos,"[chatgtp, think, tos]",False


In [78]:
reddit_df["text_contain_academic"].value_counts()

False    36511
True     12914
Name: text_contain_academic, dtype: int64

In [79]:
reddit_acad_df = reddit_df[reddit_df["text_contain_academic"] == True]

In [80]:
reddit_acad_df

,Unnamed: 0,comment_id,comment_parent_id,comment_body,subreddit,clean_text,clean_tokens,text_contain_academic
1,1,iztn0q0,t3_zj2aeu,\n\ni am so angry right now. i just wasted my...,r/ChatGPT,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c...",True
11,11,izwlu83,t3_zj2aeu,"i asked to do the opposite of that:\n\n""a titl...",r/ChatGPT,ask opposite title likely generate upvotes gar...,"[ask, opposite, title, likely, generate, upvot...",True
23,23,j0bxgq0,t3_zj2aeu,i was messing around and got it to create this...,r/ChatGPT,mess around get create masterpiecelo behold mo...,"[mess, around, get, create, masterpiecelo, beh...",True
50,50,j0c5wsj,t3_zj2aeu,dead internet theory just with more steps,r/ChatGPT,dead internet theory step,"[dead, internet, theory, step]",True
68,68,j18jqxa,t3_zj2aeu,a bunch of stuff i’ve found. \n\ncgpt can draw...,r/ChatGPT,bunch stuff ive find cgpt draw shape follow di...,"[bunch, stuff, ive, find, cgpt, draw, shape, f...",True
...,...,...,...,...,...,...,...,...
52398,16655,j4jd7lt,t1_j4j72zf,just wanted to point out that if you lost your...,r/technology,want point lose job career option pursue easy ...,"[want, point, lose, job, career, option, pursu...",True
52400,16657,j4j9fzg,t1_j4j6qvu,"yeah deepfakes, ai-assisted propaganda and god...",r/technology,yeah deepfakes ai assist propaganda god know e...,"[yeah, deepfakes, ai, assist, propaganda, god,...",True
52402,16659,j4jed9j,t1_j4jbhm0,"well, at least for the moment my job is not in...",r/technology,well least moment job jeopardy say understand ...,"[well, least, moment, job, jeopardy, say, unde...",True
52408,16665,j4kn8zu,t1_j4kk704,"that's not what i said at all.\n\nin fact, i'm...",r/technology,say fact get lot downvotes literally say multi...,"[say, fact, get, lot, downvotes, literally, sa...",True


In [81]:
reddit_acad_df = reddit_acad_df.drop(columns=['Unnamed: 0',	"comment_id",	"comment_parent_id", "subreddit",	"text_contain_academic"])

In [82]:
reddit_acad_df

,comment_body,clean_text,clean_tokens
1,\n\ni am so angry right now. i just wasted my...,angry right waste time read post sub clickbait...,"[angry, right, waste, time, read, post, sub, c..."
11,"i asked to do the opposite of that:\n\n""a titl...",ask opposite title likely generate upvotes gar...,"[ask, opposite, title, likely, generate, upvot..."
23,i was messing around and got it to create this...,mess around get create masterpiecelo behold mo...,"[mess, around, get, create, masterpiecelo, beh..."
50,dead internet theory just with more steps,dead internet theory step,"[dead, internet, theory, step]"
68,a bunch of stuff i’ve found. \n\ncgpt can draw...,bunch stuff ive find cgpt draw shape follow di...,"[bunch, stuff, ive, find, cgpt, draw, shape, f..."
...,...,...,...
52398,just wanted to point out that if you lost your...,want point lose job career option pursue easy ...,"[want, point, lose, job, career, option, pursu..."
52400,"yeah deepfakes, ai-assisted propaganda and god...",yeah deepfakes ai assist propaganda god know e...,"[yeah, deepfakes, ai, assist, propaganda, god,..."
52402,"well, at least for the moment my job is not in...",well least moment job jeopardy say understand ...,"[well, least, moment, job, jeopardy, say, unde..."
52408,"that's not what i said at all.\n\nin fact, i'm...",say fact get lot downvotes literally say multi...,"[say, fact, get, lot, downvotes, literally, sa..."


In [83]:
reddit_acad_df = reddit_acad_df.replace('\n',' ', regex=True)
reddit_acad_df = reddit_acad_df.apply(lambda x: x.str.replace(',',' '))

In [84]:
reddit_acad_df = reddit_acad_df.reset_index(drop=True)
reddit_acad_df.index.names = ['id']
reddit_acad_df = reddit_acad_df.reset_index()

In [85]:
reddit_acad_df

,id,comment_body,clean_text,clean_tokens
0,0,i am so angry right now. i just wasted my t...,angry right waste time read post sub clickbait...,NaN
1,1,"i asked to do the opposite of that: ""a title ...",ask opposite title likely generate upvotes gar...,NaN
2,2,i was messing around and got it to create this...,mess around get create masterpiecelo behold mo...,NaN
3,3,dead internet theory just with more steps,dead internet theory step,NaN
4,4,a bunch of stuff i’ve found. cgpt can draw s...,bunch stuff ive find cgpt draw shape follow di...,NaN
...,...,...,...,...
12909,12909,just wanted to point out that if you lost your...,want point lose job career option pursue easy ...,NaN
12910,12910,yeah deepfakes ai-assisted propaganda and god...,yeah deepfakes ai assist propaganda god know e...,NaN
12911,12911,well at least for the moment my job is not in...,well least moment job jeopardy say understand ...,NaN
12912,12912,that's not what i said at all. in fact i'm g...,say fact get lot downvotes literally say multi...,NaN


In [86]:
reddit_acad_df = reddit_acad_df.drop(columns=["clean_tokens"])

In [87]:
reddit_acad_df

,id,comment_body,clean_text
0,0,i am so angry right now. i just wasted my t...,angry right waste time read post sub clickbait...
1,1,"i asked to do the opposite of that: ""a title ...",ask opposite title likely generate upvotes gar...
2,2,i was messing around and got it to create this...,mess around get create masterpiecelo behold mo...
3,3,dead internet theory just with more steps,dead internet theory step
4,4,a bunch of stuff i’ve found. cgpt can draw s...,bunch stuff ive find cgpt draw shape follow di...
...,...,...,...
12909,12909,just wanted to point out that if you lost your...,want point lose job career option pursue easy ...
12910,12910,yeah deepfakes ai-assisted propaganda and god...,yeah deepfakes ai assist propaganda god know e...
12911,12911,well at least for the moment my job is not in...,well least moment job jeopardy say understand ...
12912,12912,that's not what i said at all. in fact i'm g...,say fact get lot downvotes literally say multi...


In [70]:
reddit_acad_df.to_csv("../cleaned_reddit.csv", index=False)